In [1]:
from tinysig.utils import verify_ecdsa_signature, verify_dsa_signature
from tinysig.setup import DSASetup, ECDSASetup
from tinysig.tecdsa import ThresholdSignature

# Setup network

Depending on the signature type we are aiming at (DSA or ECDSA), we can start the network with the class corresponding to that type. Let us see an example with 3 nodes and 1 client.

In [2]:
N = 3; C = 1

### DSA

To start a DSA signature, we use the class DSASetup. There are two options here:
1. Generate the DSA setup from scratch; 
2. Use a DSA setup from predefined variables.

The first option takes longer as the `generate_dsa_setup()` function has to find a primitive root for the `q` prime of the DSA setup.

In [ ]:
# Option 1
fnil = ThresholdSignature(N, C)

In [3]:
# Option 2
p = 16987220163402883416449356930313946536948708368250187300904484990592060034399925373558684845589122357155245527725130833676269318205326149268410610561367974110319706088695097181729621805806503895242356834473026604015120592348890617701675387428807728090415853765634415325555621648235338466349957683063948139664640253794461972428009207212678775162641560258829400418398089166048123240989061901894801714545511227607162820358567023001939860545346903340718565981921863209491363679301897076211852920953764568258713784702534591771430944194446014721504677270780731006777716425745362753111142293413901721847152726957943402872047 
q = 18615201011907374064080708325380633467600489307695820739772219003499; 
g = 1440750739647392583923353319762863205412412735463771135600354281498545556560554285032144083567469348458038821471561505478727536048946568600306333026876282227253717145726280535747755789389298351217147981114134445522434273687560094566805116079958307881112688486903459951003823567315217837479260063662350297462681218852749673559083125704211468000331391500202590446254295826681166987302499736009857926325072657165790463352645906484288271010829609728496608136458748019477999277575560554800692468233144862681997749241093911491601564874805253852956797072221469937174346581408575685518457073416604892562933677029344283366064
h = 2
dsa_setup = DSASetup(p, q, g, h)
fnil = ThresholdSignature(N, C, setup=dsa_setup)

**Debug mode**: the network can be set to debug mode to keep all elements sent and received throughout protocol execution.

In [4]:
fnil_debug = ThresholdSignature(N, C, setup=dsa_setup, debug=True)

### ECDSA

For ECDSA version, the can use a set of curves:

In [5]:
ECDSASetup.supported_curves()

['P-192', 'P-224', 'P-256', 'P-384', 'P-521']

In [6]:
ecdsa_setup = ECDSASetup(curve="P-256")
ecnil = ThresholdSignature(N, C, setup=ecdsa_setup)

**Debug mode**

In [7]:
ecnil_debug = ThresholdSignature(N, C, setup=ecdsa_setup, debug=True)

# Threshold (EC)DSA 

In the following demo, we emulate a network. For this reason, communication is given as a simple API and the client in the network request the network to sign some message by introducing its `id`.

In [8]:
client_id = 1

## DSA

We start by generating the secret key shared throughout the nodes. This only has to be done once.

In [9]:
# DKG protocol for DSA
fnil.distributed_key_generation_protocol(client_id)

Now, the client can ask the network to sign a message at their will without the network knowing the message.

We can start the preprocessing phase before the message is defined.

In [10]:
# Preprocessing phase (message independent)
fnil.ts_prep_protocol(client_id)

In [11]:
message = "Let me tell you a secret about Nillion."

In [12]:
# Online phase (message dependent)
fnil.ts_online_protocol(message, client_id)

We can run the above two cells as many times as we want without having to run the DKG protocol from above.

In [13]:
fnil.print_signature(client_id)

    Client(id=1,
      r=866456810140799270992901093853994083574513660692530393694283169735,
      s=17230127798322948297359259017038343422389044881055246558700591255771,
      m=Let me tell you a secret about Nillion.,
    )


#### Signature verification

The protocol already has included a verification of the signature. However, we can also verify for ourselves:

In [14]:
# Signature
r, s, m = fnil.retrieve_signature(client_id)
# Public parameters (client's public key, dsa setup)
y = fnil.clients[client_id - 1].get_open(str(client_id)+"th_client_x_pk")
q = fnil.q
p = fnil.dsa.p
g = fnil.dsa.g
# Verify
verify_dsa_signature(message, r, s, y, p, q, g)

## ECDSA

Everything works similarly to the ECDSA version. 

In [15]:
# DKG protocol for ECDSA
ecnil.distributed_key_generation_protocol(client_id)

In [16]:
# Preprocessing phase (message independent)
ecnil.ts_prep_protocol(client_id)

In [17]:
message = "This is one of many releases we are going to have in the next few months."

In [18]:
# Online phase (message dependent)
ecnil.ts_online_protocol(message, client_id)

In [19]:
# Signature
r, s, m = ecnil.retrieve_signature(client_id)
# Public parameters (client's public key, ecdsa setup)
Y = ecnil.clients[client_id - 1].get_open(str(client_id)+"th_client_x_pk")
q = ecnil.q
G = ecnil.ecdsa.G
# Verify
verify_ecdsa_signature(message, r, s, Y, q, G)

Let us see a case where the signature is not valid for some message. We give it a different message.

In [20]:
other_message = "So, are you ready for the Nillimania that is about to come?"
# Signature
r, s, m = ecnil.retrieve_signature(client_id)
# Public parameters (client's public key, ecdsa setup)
Y = ecnil.clients[client_id - 1].get_open(str(client_id)+"th_client_x_pk")
q = ecnil.q
G = ecnil.ecdsa.G
# Verify
verify_ecdsa_signature(other_message, r, s, Y, q, G)

VerifySignatureError: Signature verification failed. Signature mismatch. Abort.

### Debugging

We can also take a closer look into the network, in case we want to explore more. For that, we have available the print() function, which shows the elements owned by each party.

In [21]:
ecnil.print()

Network(N=3, q=115792089210356248762697446949407573529996955224135760342422259061068512044369,
  nodes=[
    Node(id=1,
      shares_db={
        1th_client_x_enc_sh_exp: <phe.paillier.EncryptedNumber object at 0x7fb76ffa8d90>,
        1th_client_lambda_1_enc_sh_base: <phe.paillier.EncryptedNumber object at 0x7fb76ffab7c0>,
        1th_client_lambda_2_enc_sh_base: <phe.paillier.EncryptedNumber object at 0x7fb76ffab490>,
             },
      public_keys={
        0: <PaillierPublicKey 13238cbba9>,
             },
      open_db={
        1th_client_x_sk: 28366821186158931449365894762575778610930595231666123227518790537448233657765,
        1th_client_x_pk: <Crypto.PublicKey.ECC.EccPoint object at 0x7fb7895ee380>,
        1th_client_k_r: 57171912513127058861305913686040155518728941802280615925843447425966440139137,
        1th_client_k_inv_sk: 8073490050704617529204062524429733350901213971996330479316418658370454712661,
        1th_client_gap_particle_m: 106579965961397005283454587061856

In debug mode, we have access to all elements:

In [22]:
# DKG protocol for ECDSA
ecnil_debug.distributed_key_generation_protocol(client_id)

In [23]:
# Preprocessing phase (message independent)
ecnil_debug.ts_prep_protocol(client_id)

In [24]:
message = "Stay tunned. Join the Telegram chat. Join the Discord channel. Be happy."

In [25]:
# Online phase (message dependent)
ecnil_debug.ts_online_protocol(message, client_id)

In [26]:
ecnil_debug.print()

Network(N=3, q=115792089210356248762697446949407573529996955224135760342422259061068512044369,
  nodes=[
    Node(id=1,
      shares_db={
        randomsh_node_1: 29627232355725091013240817289865842167963979835036529475876283255307412359686,
        randomsh_node_2: 25963599236838826166953673698485233994041097844090884826392497191055486845126,
        randomsh_node_3: 5085385825717084993488103744108010930455719344509659269077663467598091870118,
        random_minus_1th_client_x_sh_exp: 104102972143424312637592319339960633531467443508956898548018721560739758910052,
        1th_client_x_enc_sh_exp: <phe.paillier.EncryptedNumber object at 0x7fb76fe27b20>,
        1th_client_lambda_1_enc_sh_base: <phe.paillier.EncryptedNumber object at 0x7fb76fe26ec0>,
        1th_client_lambda_2_enc_sh_base: <phe.paillier.EncryptedNumber object at 0x7fb76fe26fb0>,
             },
      public_keys={
        0: <PaillierPublicKey 6251130f04>,
             },
      open_db={
        pow_share_node_1: 775854